In [ ]:
import math, time, json, random, os, sys
from dataclasses import dataclass
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
set_seed()

# **1. Load Data FashionMNIST**

In [ ]:
batch_size = 128
num_workers = 2

In [ ]:
# Khai báo cơ chế chuẩn hoá ảnh đầu vào
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

In [ ]:
train_dataset = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.0MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 192kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.54MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 12.7MB/s]


# **2. Chia data thành các batch - truyền vào DataLoader**

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=num_workers, pin_memory=True)

In [ ]:
classes = train_dataset.classes
classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

# **Expert**

+ Mỗi "expert" trong kiến trúc MOE là một mô hình nhỏ, độc lập
+ "expert" được thiết kế đơn giản như một lớp Feed Forward Neural Network
+ Mỗi "expert" có khả năng xử lý input một cách chuyên biệt

In [ ]:
class Expert(nn.Module):
    def __init__(self, in_dim, hidden_dim, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

# **Router**

+ Đây là phần quan trọng nhất, quyết định expert nào sẽ xử lý input
+ Phần Router, sử dụng top-k routing (chọn k expert có phần trăm cao nhất), nhưng thêm noisy (nhiễu) để tránh vấn đề "expert collapse" - nơi một expert handle toàn bộ input, dẫn đến các expert khác không được huấn luyện
+ Noise chỉ được thêm trong quá trình training để khuyến khích router sử dụng đa dạng các expert, nhưng trong quá trình inference (test) thì không thêm
+ noisy_std là chỉ số cho phép điều chỉnh mức độ noise (nhiễu)

In [ ]:
class TopKRouter(nn.Module):
    def __init__(self, in_dim, num_experts, noisy_std=1.0):
        super().__init__()
        self.w_g = nn.Linear(in_dim, num_experts) # Dùng để tính ra tỷ lệ phần trăm expert nào sẽ được sử dụng (gating logits)
        self.noisy_std = noisy_std

    def forward(self, x):
        logits = self.w_g(x)
        if self.training and self.noisy_std > 0:
            noise = torch.randn_like(logits) * self.noisy_std
            logits += noise
        return logits

# **Sparse MOE**

In [ ]:
class SparseMOE(nn.Module):
    def __init__(self, in_dim, hidden_dim, num_experts=8, top_k=2, dropout=0.1, noisy_std=1.0):
        super().__init__()
        self.num_experts = num_experts
        self.top_k = top_k
        self.router = TopKRouter(in_dim, num_experts, noisy_std=noisy_std)
        self.experts = nn.ModuleList([Expert(in_dim, hidden_dim, dropout=dropout) for _ in range(num_experts)])
        self.combine = nn.Linear(hidden_dim, hidden_dim, bias=False)

    def forward(self, x):
        # 1. Tính logits [B, experts] - đại diện cho điểm số (scores) cho từng expert đối với mỗi sample trong batch
        # Batch = 10, expert = 8
        '''
        [
          [0.3, 0.1, 0.2, ..., 0.05],
          [],
          [],
          [],
          ...,
          []
        ]
        '''
        logits = self.router(x)
        # Sử dụng torch.topk để lấy top-k giá trị lớn nhất (top_vals: [B, top_k])
        topk_vals, topk_idx = torch.topk(logits, self.top_k, dim=-1)
        # Áp dụng softmax lên top_vals để chuyển thành xác suất
        gates = F.softmax(topk_vals, dim=-1)


        B, K = gates.shape
        hidden = None

        # Lặp qua từng vị trí K trong Top-K
        for k in range(K):
            idx = topk_idx[:, k]
            gate = gates[:, k].unsqueeze(-1)

            # List để thu thập output từ các expert
            chunks = []
            for expert in range(self.num_experts):
                # Tạo mask [Batch size, 1] - xác định xem trong số các expert thì expert nào được chọn
                mask = (idx == expert).float().unsqueeze(-1)
                if mask.sum() == 0:
                    continue

                # Tính output của expert cho toàn bộ batch
                out_expert = self.experts[expert](x) * mask
                # Thu thập output
                chunks.append(out_expert)

            if len(chunks) == 0:
                continue

            out_k = torch.stack(chunks, dim=0).sum(dim=0)
            out_k = out_k * gate
            hidden = out_k if hidden is None else hidden + out_k

        hidden = self.combine(hidden)
        return hidden

# **CNNFeatureExtractor**

In [ ]:
class CNNFeatureExtractor(nn.Module):
    def __init__(self, feat_dim=128):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.proj = nn.Linear(128, 128)

    def forward(self, x):
        z = self.conv(x)
        z = z.view(z.size(0), -1)
        z = self.proj(z)
        return z

# **Base Line Model**

In [ ]:
class BaselineModel(nn.Module):
    def __init__(self, feat_dim=128, hidden=64, num_classes=10, dropout=0.2):
        super().__init__()
        self.backbone = CNNFeatureExtractor(feat_dim)
        self.head = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(feat_dim, hidden),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(hidden, num_classes)
        )

    def forward(self, x):
        f = self.backbone(x)
        return self.head(f)

# **MOE model**

In [ ]:
class MOEModel(nn.Module):
    def __init__(self, feat_dim=128, hidden_dim=64, num_classes=10, num_experts=8, top_k=2, dropout=0.2, noisy_std=1.0):
        super().__init__()
        self.backbone = CNNFeatureExtractor(feat_dim=feat_dim)
        self.moe = SparseMOE(in_dim=feat_dim, hidden_dim=hidden_dim, num_experts=num_experts, top_k=top_k, dropout=dropout, noisy_std=noisy_std)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        f = self.backbone(x)
        h = self.moe(f)
        return self.fc(h)

# **Huấn luyện và so sánh**

In [ ]:
from dataclasses import dataclass

@dataclass
class TrainConfig:
    epochs: int = 5
    lr: float = 1e-3
    weight_decay: float = 0.0
    grad_clip: float = 1.0

def accuracy(pred, target):
    return (pred.argmax(dim=1) == target).float().mean().item()

def train_one_epoch(model, loader, opt, cfg: TrainConfig):
    model.train()
    total_loss, total_acc, total_n = 0.0, 0.0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        opt.zero_grad()
        logits = model(x)
        loss = F.cross_entropy(logits, y)
        loss.backward()
        if cfg.grad_clip is not None:
            nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip)
        opt.step()
        bs = x.size(0)
        total_loss += loss.item() * bs
        total_acc  += accuracy(logits.detach(), y) * bs
        total_n    += bs
    return total_loss/total_n, total_acc/total_n

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    total_loss, total_acc, total_n = 0.0, 0.0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = F.cross_entropy(logits, y)
        bs = x.size(0)
        total_loss += loss.item() * bs
        total_acc  += accuracy(logits, y) * bs
        total_n    += bs
    return total_loss/total_n, total_acc/total_n

In [ ]:
cfg = TrainConfig(epochs=5, lr=1e-3, weight_decay=0.0, grad_clip=1.0)

baseline = BaselineModel(feat_dim=128, hidden=64, num_classes=10, dropout=0.2).to(device)
moe = MOEModel(feat_dim=128, hidden_dim=64, num_classes=10, num_experts=8, top_k=2, dropout=0.2, noisy_std=1.0).to(device)

opt_base = torch.optim.Adam(baseline.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
opt_moe  = torch.optim.Adam(moe.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)

In [ ]:
hist = {"baseline": {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": [],
                     "train_time": [], "test_time": []},
        "moe": {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": [],
               "train_time": [], "test_time": []}}

print("== Train Baseline ==")
start_time_base = time.time()  # Bắt đầu đo tổng thời gian Baseline
for epoch in range(1, cfg.epochs+1):
    # Đo thời gian train một epoch
    epoch_start = time.time()
    tl, ta = train_one_epoch(baseline, train_loader, opt_base, cfg)
    train_time = time.time() - epoch_start

    # Đo thời gian evaluate một epoch
    epoch_start = time.time()
    vl, va = evaluate(baseline, test_loader)
    test_time = time.time() - epoch_start

    hist["baseline"]["train_loss"].append(tl)
    hist["baseline"]["train_acc"].append(ta)
    hist["baseline"]["test_loss"].append(vl)
    hist["baseline"]["test_acc"].append(va)
    hist["baseline"]["train_time"].append(train_time)
    hist["baseline"]["test_time"].append(test_time)

    print(f"[Baseline][Epoch {epoch:02d}] train_loss={tl:.4f} train_acc={ta:.4f} "
          f"| test_loss={vl:.4f} test_acc={va:.4f} "
          f"| train_time={train_time:.2f}s test_time={test_time:.2f}s")

total_time_base = time.time() - start_time_base
print(f"\nTổng thời gian huấn luyện Baseline: {total_time_base:.2f} giây")

print("\n== Train MoE ==")
start_time_moe = time.time()  # Bắt đầu đo tổng thời gian MoE
for epoch in range(1, cfg.epochs+1):
    epoch_start = time.time()
    tl, ta = train_one_epoch(moe, train_loader, opt_moe, cfg)
    train_time = time.time() - epoch_start

    epoch_start = time.time()
    vl, va = evaluate(moe, test_loader)
    test_time = time.time() - epoch_start

    hist["moe"]["train_loss"].append(tl)
    hist["moe"]["train_acc"].append(ta)
    hist["moe"]["test_loss"].append(vl)
    hist["moe"]["test_acc"].append(va)
    hist["moe"]["train_time"].append(train_time)
    hist["moe"]["test_time"].append(test_time)

    print(f"[MoE][Epoch {epoch:02d}] train_loss={tl:.4f} train_acc={ta:.4f} "
          f"| test_loss={vl:.4f} test_acc={va:.4f} "
          f"| train_time={train_time:.2f}s test_time={test_time:.2f}s")

total_time_moe = time.time() - start_time_moe
print(f"\nTổng thời gian huấn luyện MoE: {total_time_moe:.2f} giây")

# === Tóm tắt so sánh thời gian ===
avg_train_base = sum(hist["baseline"]["train_time"]) / cfg.epochs
avg_test_base  = sum(hist["baseline"]["test_time"])  / cfg.epochs
avg_train_moe  = sum(hist["moe"]["train_time"])     / cfg.epochs
avg_test_moe   = sum(hist["moe"]["test_time"])      / cfg.epochs

print("\n=== SO SÁNH THỜI GIAN TRUNG BÌNH MỖI EPOCH ===")
print(f"{'Model':<10} | {'Train (s)':<12} | {'Test (s)':<12} | {'Total (s)':<10}")
print("-" * 50)
print(f"{'Baseline':<10} | {avg_train_base:10.2f}   | {avg_test_base:10.2f}   | {total_time_base:8.2f}")
print(f"{'MoE':<10}     | {avg_train_moe:10.2f}   | {avg_test_moe:10.2f}   | {total_time_moe:8.2f}")

# Lưu lịch sử (bây giờ có thêm thời gian)
with open("fashionmnist_moe_history.json", "w") as f:
    json.dump(hist, f, indent=2)
print("\nĐã lưu lịch sử (bao gồm thời gian) vào fashionmnist_moe_history.json")

== Train Baseline ==
[Baseline][Epoch 01] train_loss=1.1004 train_acc=0.5778 | test_loss=0.7354 test_acc=0.7280 | train_time=15.54s test_time=2.20s
[Baseline][Epoch 02] train_loss=0.6654 train_acc=0.7476 | test_loss=0.5558 test_acc=0.7874 | train_time=15.26s test_time=1.89s
[Baseline][Epoch 03] train_loss=0.5404 train_acc=0.8005 | test_loss=0.4962 test_acc=0.8191 | train_time=14.21s test_time=1.91s
[Baseline][Epoch 04] train_loss=0.4724 train_acc=0.8296 | test_loss=0.4323 test_acc=0.8461 | train_time=14.19s test_time=1.95s
[Baseline][Epoch 05] train_loss=0.4173 train_acc=0.8491 | test_loss=0.3982 test_acc=0.8557 | train_time=14.22s test_time=2.91s

Tổng thời gian huấn luyện Baseline: 84.28 giây

== Train MoE ==
[MoE][Epoch 01] train_loss=1.2236 train_acc=0.5166 | test_loss=0.8420 test_acc=0.6691 | train_time=17.65s test_time=2.03s
[MoE][Epoch 02] train_loss=0.7731 train_acc=0.7004 | test_loss=0.6544 test_acc=0.7528 | train_time=18.01s test_time=2.33s
[MoE][Epoch 03] train_loss=0.6482 t